In [ ]:
# So Colab can use the python files...
!git clone --single-branch -b develop https://github.com/muellevin/Studienarbeit.git
%cd Studienarbeit/Detection_training

# 1. Setup Paths

In [ ]:
import os
import Tensorflow.scripts.Paths as pp

CUSTOM_MODEL_NAME = 'raccoonModel' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

paths = pp.paths
paths.setup_paths()

CHECKPOINT_PATH = os.path.join(paths.MODEL_PATH, CUSTOM_MODEL_NAME)
OUTPUT_PATH = os.path.join(paths.MODEL_PATH, CUSTOM_MODEL_NAME, 'export')
os.makedirs(CHECKPOINT_PATH, exist_ok=True)
os.makedirs(OUTPUT_PATH, exist_ok=True)

PIPELINE_CONFIG = os.path.join(paths.MODEL_PATH, CUSTOM_MODEL_NAME, 'pipeline.config')

# 2. Download pretrained Tensorflow Model and install TFOD

In [ ]:
if os.name=='nt':
    !pip install wget
    import wget

In [ ]:
# Download TensorFlow Object detection library if not available
if not os.path.exists(os.path.join(paths.APIMODEL_PATH, 'research', 'object_detection')):
    command = "git clone https://github.com/tensorflow/models {}".format(paths.APIMODEL_PATH)

    !{command}

Install TFOD (TensorFlow Object Detection)

In [ ]:
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths.PROTOC_PATH}
    !cd {paths.PROTOC_PATH} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths.PROTOC_PATH, 'bin'))
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

Verify that everything is setup correctly.  
The script should execute with 'OK (skipped=1)'

In [ ]:
%pip install tensorflow==2.9
%pip install tensorflow_addons
%pip install gin-config

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths.APIMODEL_PATH, 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [ ]:
import object_detection

In [ ]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths.PRETRAINED_MODEL_PATH}
    !cd {paths.PRETRAINED_MODEL_PATH} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths.PRETRAINED_MODEL_PATH}
    !cd {paths.PRETRAINED_MODEL_PATH} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

# 3. Create Label Map

In [ ]:
with open(pp.LABELMAP, 'w') as f:
    for label in pp.LABELS:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 4. Upload dataset to Colab

Upload the dataset to google drive

In [ ]:
GOOGLE_DRIVE_PATH = '/content/drive/MyDrive/Studienarbeit'

if os.name =='posix':
    from google.colab import drive

    drive.mount('/content/drive')

    import shutil
    shutil.copy(os.path.join(GOOGLE_DRIVE_PATH, pp.DATASET_NAME), paths.IMAGE_PATH)

Now your google drive is connected and you have to copy the dataset into the image directory

Verify that dataset is available

In [ ]:
if not os.path.isfile(pp.DATASET):
  print('No dataset available')
else:
  print('Dataset available')

# 5. Create TF records

In [ ]:
if os.path.exists(pp.DATASET):
  !tar -zxvf {pp.DATASET}

In [ ]:
!python {pp.TF_RECORD_SCRIPT} -img_path {paths.TRAINSET_PATH} -output_path {pp.TRAINSET_RECORD_PATH}
!python {pp.TF_RECORD_SCRIPT} -img_path {paths.TESTSET_PATH} -output_path {pp.TESTSET_RECORD_PATH}

# 6. Copy Model Config to Training Folder

In [ ]:
if os.name =='posix':
    !cp {os.path.join(paths.PRETRAINED_MODEL_PATH, PRETRAINED_MODEL_NAME, 'pipeline.config')} {CHECKPOINT_PATH}
if os.name == 'nt':
    !copy {os.path.join(paths.PRETRAINED_MODEL_PATH, PRETRAINED_MODEL_NAME, 'pipeline.config')} {CHECKPOINT_PATH}

# 7. Update Config for Transfer Learning

In [ ]:
import tensorflow as tf
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(PIPELINE_CONFIG, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

pipeline_config.model.ssd.num_classes = len(pp.LABELS)
pipeline_config.train_config.batch_size = 16
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths.PRETRAINED_MODEL_PATH, PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= pp.LABELMAP
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [pp.TRAINSET_RECORD_PATH]
pipeline_config.eval_input_reader[0].label_map_path = pp.LABELMAP
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [pp.TESTSET_RECORD_PATH]

config_text = text_format.MessageToString(pipeline_config)

with tf.io.gfile.GFile(PIPELINE_CONFIG, "wb") as f:
    f.write(config_text)

# 8. Train the model

In [ ]:
TRAINING_SCRIPT = os.path.join(paths.APIMODEL_PATH, 'research', 'object_detection', 'model_main_tf2.py')
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, CHECKPOINT_PATH, PIPELINE_CONFIG)
print(command)

In [ ]:
!{command}

# 9. Freezing the Graph

In [ ]:
FREEZE_SCRIPT = os.path.join(paths.APIMODEL_PATH, 'research', 'object_detection', 'exporter_main_v2.py ')
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT , PIPELINE_CONFIG, CHECKPOINT_PATH, OUTPUT_PATH)

In [ ]:
!{command}

# 10. Zip and Download Model in Colab

In [ ]:
!zip -r {os.path.join(GOOGLE_DRIVE_PATH, CUSTOM_MODEL_NAME + '.zip')} {CHECKPOINT_PATH}